In [1]:
import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
# import torch
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline

c:\Users\Nick\AppData\Local\pypoetry\Cache\virtualenvs\url-classification-nbz3569H-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
df_huffpo = pd.read_parquet("../../data/raw/news_categories.parquet", columns=["link", "category"])

In [17]:
df_huffpo["slug"] = df_huffpo.link.apply(lambda x: x.split("entry/")[-1].split("_")[0].replace("-", " "))

In [24]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

c:\Users\Nick\AppData\Local\pypoetry\Cache\virtualenvs\url-classification-nbz3569H-py3.12\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nick\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Nick\AppData\Local\pypoetry\Cache\virtualenv

In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
df_huffpo["preprocessed"] = df_huffpo.slug.apply(tokenizer)

In [29]:
cats_enum = list(enumerate(set(df_huffpo.category)))
id2label = {i[0]: i[1] for i in cats_enum}
label2id = {i[1]: i[0] for i in cats_enum}

In [32]:
metrics = evaluate.combine([
    evaluate.load("precision"),
    evaluate.load("recall")
])
def compute_metrics(eval_pred):
    predictions, labels=eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metrics.compute(predictions, references=labels, average="weighted")